## EECS 731 Project 2

### Step 0: Import relevant packages

In [1]:
from __future__ import print_function

import os
#import sys
import time
import pandas as pd
import numpy  as np
#import nltk
import sklearn
import networkx as nx
import numpy  as np
import pandas as pd
from pandas import compat
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score,r2_score
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.tokenize.treebank import TreebankWordDetokenizer
import seaborn as sns; sns.set()

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Packages Loaded")

Packages Loaded


### Step 1: Pre requisite functions

#### This function load the data from the given path and finename

In [2]:
#Function to load the data
def loadData(path,filename):
    try:
             files = os.listdir(path)
             for f in files:
                 if f == filename:
                     data = pd.read_csv(os.path.join(path,f))
                     return data
            
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This function derives the shape of the dataset and returns the feature and target columns

In [3]:
#Function to explore the data
def exploreData(data):
    try:
           #Total number of records                                  
           rows = data.shape[0]
           cols = data.shape[1]    
           
           #separate features and target
           drop_col = ['target']
           features = data.drop(drop_col, axis = 1)
           target = data[drop_col]
          
           # Print the results
           print ("-----------------------------------------------------------------------")
           print ("Total number of records: {}".format(rows))
           print ("Total number of features: {}".format(cols))
           print ("-----------------------------------------------------------------------")
           
           return features,target
           
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This function computes the percentage of missing values per each column in the given dataset

In [4]:
def missingValues(data):
    try:
           # Total missing values
           mis_val = data.isnull().sum()
         
           # Percentage of missing values
           mis_val_percent = 100 * mis_val / len(data)
           
           # Make a table with the results
           mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
           
           # Rename the columns
           mis_val_table_ren_columns = mis_val_table.rename(
           columns = {0 : 'Missing Values', 1 : '% of Total Values'})
           mis_val_table_ren_columns.head(4 )
           # Sort the table by percentage of missing descending
           misVal = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
                   '% of Total Values', ascending=False).round(1)
                     
           return misVal, mis_val_table_ren_columns

    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method token the input string and return the sum of the counts of each word in the corpus

In [5]:
def tokenString(sen,fdist,stop_w):
    try:
        tokens = word_tokenize(sen)
        sen = [w for w in tokens if not w in stop_w]
        sen = [w for w in sen if w.isalpha()]
        #sen_f = TreebankWordDetokenizer().detokenize(sen)
        wordcnt = 0
        for w in sen:
            wordcnt = wordcnt + fdist[w]
        #print(tokens)
        #print(sen_f)
        #sys.stdout.write('.'); sys.stdout.flush();
        
        return wordcnt 
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method scale the numerical features and label encoding for categorical features 

In [6]:
def transformData(features,target):
    try:
        
#        skewed = ['rain']
#        features_log_transformed = pd.DataFrame(data = features)
#        features_log_transformed[skewed] = features[skewed].apply(lambda x: np.log(x + 1))
        
        scaler = MinMaxScaler() # default=(0, 1)
        numerical = ['PlayerLine','LineCount']
        features_log_minmax_transform = pd.DataFrame(data = features)
        features_log_minmax_transform[numerical] = scaler.fit_transform(features[numerical])
                
        # TODO: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
        #features_final = pd.get_dummies(features_log_minmax_transform)
        enc = LabelEncoder()
        features_log_minmax_transform['Play'] = enc.fit_transform(features_log_minmax_transform['Play'])
        target = enc.fit_transform(target)
        features_final = features_log_minmax_transform
        # Print the number of features after one-hot encoding
        #encoded = list(features_final.columns)
        #print "{} total features after one-hot encoding.".format(len(encoded))
         
        return features_final, target
        
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method splits data in to training and test data

In [7]:
#split the data in to train and test data
def splitData(features,target,testsize):
    try:
        # Split the 'features' and 'income' data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target, 
                                                    test_size = testsize, 
                                                    random_state = 1)

        # Show the results of the split
        print ("Features training set has {} samples.".format(X_train.shape[0]))
        print ("Features testing set has {} samples.".format(X_test.shape[0]))
        print ("Target training set has {} samples.".format(y_train.shape[0]))
        print ("Target testing set has {} samples.".format(y_test.shape[0]))
        print ("-----------------------------------------------------------------------")
        return X_train, X_test, y_train, y_test
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method implements the SVM classifier

In [8]:
def svmClassifier(X_train, X_test, y_train, y_test):
    try:
        #Decision tree classifier
        #learner = DecisionTreeClassifier(criterion=method, max_depth=depth, random_state=1)
        clf = svm.SVC(random_state=0)
        params = {'gamma':[0.001]}
        #params = {'criterion':['gini','entropy'], 'max_depth' : np.array([6,7,8])}
         
        scoring_fnc = make_scorer(fbeta_score,average='micro',beta=0.5)
        learner = GridSearchCV(clf,params,scoring=scoring_fnc)
        results = {}
         
        start_time = time.clock()
        grid = learner.fit(X_train,y_train)
         
        end_time = time.clock()
        results['train_time'] = end_time - start_time
        clf_fit_train = grid.best_estimator_
        start_time = time.clock()
        clf_predict_train = clf_fit_train.predict(X_train)
        clf_predict_test = clf_fit_train.predict(X_test)
        end_time = time.clock()
        results['pred_time'] = end_time - start_time  
         
        results['acc_train'] = accuracy_score(y_train, clf_predict_train)
        results['acc_test']  = accuracy_score(y_test, clf_predict_test)
#        results['f_train']   = fbeta_score(y_train, clf_predict_train, average='micro', beta=1)
#        results['f_test']    = fbeta_score(y_test, clf_predict_test, average='micro', beta=1.5)
        
        return results,clf_fit_train      
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method implements Decision tree classifer

In [9]:
def decTree(X_train, y_train, X_test, y_test, method, depth):
    try:
        #Decision tree classifier
        #learner = DecisionTreeClassifier(criterion=method, max_depth=depth, random_state=1)
        clf = DecisionTreeClassifier(random_state=0)
        params = {'max_depth':[depth],'criterion':[method]}
        #params = {'criterion':['gini','entropy'], 'max_depth' : np.array([6,7,8])}

        scoring_fnc = make_scorer(fbeta_score,average='micro',beta=0.5)
        learner = GridSearchCV(clf,params,scoring=scoring_fnc)
        results = {}
         
        start_time = time.clock()
        grid = learner.fit(X_train,y_train)
        
        end_time = time.clock()
        results['train_time'] = end_time - start_time
        clf_fit_train = grid.best_estimator_
        start_time = time.clock()
        clf_predict_train = clf_fit_train.predict(X_train)
        clf_predict_test = clf_fit_train.predict(X_test)
        end_time = time.clock()
        results['pred_time'] = end_time - start_time  
         
        results['acc_train'] = accuracy_score(y_train, clf_predict_train)
        results['acc_test']  = accuracy_score(y_test, clf_predict_test)
        results['f_train']   = fbeta_score(y_train, clf_predict_train, average='micro', beta=1)
        results['f_test']    = fbeta_score(y_test, clf_predict_test, average='micro', beta=1.5)
        
        return results,clf_fit_train      
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

#### This method implements Naive Bayes classifier

In [10]:
def naiveBayes(X_train, y_train, X_test, y_test):
    try:
        #Decision tree classifier
        #learner = DecisionTreeClassifier(criterion=method, max_depth=depth, random_state=1)
        clf = MultinomialNB()
        params = {}

        scoring_fnc = make_scorer(fbeta_score,average='micro',beta=0.5)
        learner = GridSearchCV(clf,params,scoring=scoring_fnc)
        results = {}
         
        start_time = time.clock()
        grid = learner.fit(X_train,y_train)
        
        end_time = time.clock()
        results['train_time'] = end_time - start_time
        clf_fit_train = grid.best_estimator_
        start_time = time.clock()
        clf_predict_train = clf_fit_train.predict(X_train)
        clf_predict_test = clf_fit_train.predict(X_test)
        end_time = time.clock()
        results['pred_time'] = end_time - start_time  
         
        results['acc_train'] = accuracy_score(y_train, clf_predict_train)
        results['acc_test']  = accuracy_score(y_test, clf_predict_test)
        results['f_train']   = fbeta_score(y_train, clf_predict_train, average='micro', beta=1)
        results['f_test']    = fbeta_score(y_test, clf_predict_test, average='micro', beta=1.5)
        
        return results,clf_fit_train      
    except Exception as ex:
           print ("-----------------------------------------------------------------------")
           template = "An exception of type {0} occurred. Arguments:\n{1!r}"
           message = template.format(type(ex).__name__, ex.args)
           print (message)

##### Load the shake spear data in to a dataframe

In [18]:
compat.PY3 = True
print ("-----------------------------------------------------------------------")
print('The scikit-learn version is {}.'.format(sklearn.__version__))

#load functions from 
#from projectFunctions import loadData, exploreData, missingValues, tokenString, transformData

path = r'C:\Users\pmspr\Documents\HS\MS\Sem 3\EECS 731\Week 4\HW\Git\EECS-731-Project-2\Data'
filename = "Shakespeare_data.csv"
data = loadData(path,filename)
drop_col = ['Dataline','PlayerLinenumber','ActSceneLine']
data = data.drop(drop_col, axis = 1)
data.rename(columns={'Player':'target'},inplace=True)
#print(data.columns)

-----------------------------------------------------------------------
The scikit-learn version is 0.20.3.


##### Compute the missing values per feature

In [19]:
print ("----------------------Shakespear Play data-----------------------------")
features, target = exploreData(data)
misVal, mis_val_table_ren_columns = missingValues(data)

# Print some summary information
print ("Columns that have missing values:" + str(misVal.shape[0]))
print ("-----------------------------------------------------------------------")
print(mis_val_table_ren_columns.head(20))

----------------------Shakespear Play data-----------------------------
-----------------------------------------------------------------------
Total number of records: 111396
Total number of features: 3
-----------------------------------------------------------------------
Columns that have missing values:1
-----------------------------------------------------------------------
            Missing Values  % of Total Values
Play        0               0.000000         
target      7               0.006284         
PlayerLine  0               0.000000         


Analysis:
We can see only target feature has missing values. We can remove the rows with missing values.

In [20]:
#Remove rows with missing target values
ind = data[data['target'].isnull()].index.tolist()
data = data.drop(index=ind, axis=0)

#Compute features to add value
line_count = data.groupby(['Play','target'], as_index=False).count()
line_count.rename(columns={'PlayerLine':'LineCount'},inplace=True)

#merge the group by counts to original dataframe
data = pd.merge(data,line_count,on=['Play','target'], how='inner')

#Remove rows with line count <=200
ind = data[data['LineCount'] <= 300].index.tolist()
data = data.drop(index=ind, axis=0)

Analysis:
- Remove the rows having missing values in target feature column
- Compute the number of lines for a 'Play' and 'Player' combination. This can act as a feature that can add some domain value.
- Merge the counts with main table and remove rows having line counts <200. This is needed because, dataset is huge and SVM classifier is clocking more time.

##### Compute the importance of a class (Player) in the target feature

In [21]:
#Find the important players in a play using network
#network graph
g = nx.Graph()
g = nx.from_pandas_edgelist(data,source='Play',target='target')
 
#Get importance by computing Degree of centrality
col = ['Degree','PageRank','Name']
doc = pd.DataFrame(columns = col)
doc['Degree'] = nx.degree_centrality(g).values()
doc['PageRank'] = nx.pagerank(g).values()
doc['Name'] = nx.degree_centrality(g).keys()

#Extract only importance of players
doc_target = doc[doc['Name'].isin(data['target'].unique().tolist())]
doc_target = doc_target.drop(['PageRank'], axis = 1)
doc_target.rename(columns={'Name':'target'},inplace=True)
data = pd.merge(data,doc_target,on=['target'], how='inner')

Analysis:
- Degree of centrality can be seen as the importance of a class. Using network graph, we compute the centrality, Pagerank of the Play and Players.
- Extract the centrality of the target (player) feature and merge with in to the main dataframe.

##### Text feature analysis

In [22]:
t = []
_ = data['PlayerLine'].apply(lambda x: t.append(x))
corpus = ' '.join(t)
#corpus = ' '.join(str(i) for i in t)
stop_w = set(stopwords.words('english'))
tokens = word_tokenize(corpus)
sen = [w for w in tokens if not w in stop_w]
corpus = [w for w in sen if w.isalpha()]
fdist=FreqDist(corpus)

#tokenize the strig
#Compute the frequency of words in a sentence 
data['PlayerLine'] = data['PlayerLine'].apply(lambda x: tokenString(x,fdist,stop_w))
data.to_csv('test.csv',index=False)

Analysis:
- We use nltk library to do perform text analysis.
- use tokenizer to divide a sentence in to words. Remove the stop words. Stop words are the ones which do not carry much meaning. 
- Compute a corpus of text from all the lines in the dataset.
- Compute the counts of each word in the sentence against the corpus. 
- This can add as another feature that can add domain to our dataset

In [23]:
features, target = exploreData(data)
features_final, target_final = transformData(features, target)

#Split the data with test size = 30
#from projectFunctions import splitData,svmClassifier
X_train, X_test, y_train, y_test = splitData(features_final, target_final, 0.3)

-----------------------------------------------------------------------
Total number of records: 50740
Total number of features: 5
-----------------------------------------------------------------------
Features training set has 35518 samples.
Features testing set has 15222 samples.
Target training set has 35518 samples.
Target testing set has 15222 samples.
-----------------------------------------------------------------------


Analysis:
- We split our data in to train and test data with ratio of 70% training data and 30% test data. 
- As we have removed few rows with less line count, we can see the difference in split data sizes and original data.

##### Implement Classifiers

In [24]:
results,learner = svmClassifier(X_train, X_test, y_train, y_test)

print ("Times for Training, Prediction: %.5f, %.5f" %(results['train_time'], results['pred_time']))    
print ("Accuracy for Training, Test sets: %.5f, %.5f" %(results['acc_train'], results['acc_test']))     
print ("-----------------------------------------------------------------------")

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Times for Training, Prediction: 305.31733, 199.70584
Accuracy for Training, Test sets: 0.30706, 0.29950
-----------------------------------------------------------------------


Analysis:
- In SVM classifer, we can see the accuracy of 30%. 
- As we have less features, we added extra two features and used 'rbf' kernel for polynomial classificatio. I believe, our data is not linearly separable. 
- As we have considerable data, so SVM classifier took considerable time to learn.

In [25]:
results,learner = decTree(X_train, y_train, X_test, y_test, 'gini', 13)
 
print ("Times for Training, Prediction: %.5f, %.5f" %(results['train_time'], results['pred_time']))     
print ("Accuracy for Training, Test sets: %.5f, %.5f" %(results['acc_train'], results['acc_test']))     
print ("-----------------------------------------------------------------------")

Times for Training, Prediction: 0.28489, 0.01844
Accuracy for Training, Test sets: 0.30382, 0.30318
-----------------------------------------------------------------------


Analysis:
- In the above code, we implemented Decision tree classifier with 'gini' method with depth 13.
- We can see an accuracy of 30% like SVM but within a very considerably less time.
- I believe Decision tree is overfitting, considering the fact of the depth used and number of features. 

In [26]:
results,learner = naiveBayes(X_train, y_train, X_test, y_test)
 
print ("Times for Training, Prediction: %.5f, %.5f" %(results['train_time'], results['pred_time']))    
print ("Accuracy for Training, Test sets: %.5f, %.5f" %(results['acc_train'], results['acc_test']))    
print ("-----------------------------------------------------------------------")

Times for Training, Prediction: 0.24602, 0.03812
Accuracy for Training, Test sets: 0.08258, 0.08461
-----------------------------------------------------------------------


Analysis:
- In the above cell, we implemented multinomial naive bayes classifer. I think, naive bayes is used for text analysis.
- We can see the accuracy is very less compared to the SVM and decision tree.

Conclusion:
- By looking at the data, we can see number of features is less. So we did text analysis to come up with extra two features that can help with modelling.
- Out of all 3 classifers, SVM and Decision tree performed to an extent. I think SVM is good on classifying our data. 